# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [45]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [46]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/dmax37100/Desktop/Project 2 Cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [47]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [48]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [49]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [50]:
# TO-DO: Create a Keyspace 

try:
    session.execute("""
    CREATE KEYSPACE sparkify_events
    WITH replication = 
    {'class':'SimpleStrategy', 'replication_factor':'1'} 
    AND durable_writes = 'true';
    """)

except Exception as e:
    print(e)

Keyspace 'sparkify_events' already exists


#### Set Keyspace

In [51]:
# TO-DO: Set KEYSPACE to the keyspace specified above

try:
    session.set_keyspace('sparkify_events')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [52]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# DROP TABLE
session.execute("DROP TABLE IF EXISTS music_app_session")

# CREATE TABLE
musicApp_table_query = (""" CREATE TABLE IF NOT EXISTS music_app_session
                        (artist text, songTitle text, songLength float, userFirstName text, userLastName text,
                        sessionId int, itemInSession int, PRIMARY KEY(sessionId, itemInSession))
                        """)

# EXECUTE QUERY
try:
    session.execute(musicApp_table_query)
except Exception as e:  print(e)
                    

In [53]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        musicApp_table_query  = ("""INSERT INTO music_app_session
                                (artist, songTitle, songLength, userFirstName, userLastName, sessionId,
                                itemInSession)
                                VALUES (%s, %s, %s, %s, %s, %s, %s)
                                """)
        
        
        
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(musicApp_table_query, (line[0], line[9],float(line[5]), line[1],line[4],int(line[8]),int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [54]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

musicApp_table_query = """
SELECT artist, songTitle, songLength
FROM music_app_session
WHERE sessionId = 338 AND itemInSession = 4
"""


try:
    rows = session.execute(musicApp_table_query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Faithless', songtitle='Music Matters (Mark Knight Dub)', songlength=495.30731201171875)


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [55]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# DROP TABLE
session.execute("DROP TABLE IF EXISTS music_app_user")

# CREATE TABLE
musicApp_table_query = (""" CREATE TABLE IF NOT EXISTS music_app_user
                        (artist text, songTitle text, userId int, userFirstName text, userLastName text,
                        sessionId int, itemInSession int, PRIMARY KEY ((userId, sessionId), itemInSession))
                        """)

# EXECUTE QUERY
try:
    session.execute(musicApp_table_query)
except Exception as e:  print(e)
                    

In [56]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        musicApp_table_query  = ("""INSERT INTO music_app_user 
                                (artist, songTitle, userId, userFirstName, userLastName,
                                sessionId, itemInSession)
                                VALUES (%s, %s, %s, %s, %s, %s, %s)
                                """)
        
        
        
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(musicApp_table_query, (line[0], line[9],int(line[10]), line[1],line[4],int(line[8]),int(line[3])))

In [57]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

musicApp_table_query = """
SELECT artist, songTitle, userFirstName,userLastName, sessionId
FROM music_app_user
WHERE userId = 10 AND sessionId = 182
ORDER BY itemInSession
"""


try:
    rows = session.execute(musicApp_table_query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Down To The Bone', songtitle="Keep On Keepin' On", userfirstname='Sylvie', userlastname='Cruz', sessionid=182)
Row(artist='Three Drives', songtitle='Greece 2000', userfirstname='Sylvie', userlastname='Cruz', sessionid=182)
Row(artist='Sebastien Tellier', songtitle='Kilometer', userfirstname='Sylvie', userlastname='Cruz', sessionid=182)
Row(artist='Lonnie Gordon', songtitle='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', userfirstname='Sylvie', userlastname='Cruz', sessionid=182)


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [58]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# DROP TABLE
session.execute("DROP TABLE IF EXISTS music_app_username")

# CREATE TABLE
musicApp_table_query = (""" CREATE TABLE IF NOT EXISTS music_app_username
                        (songTitle text, userId int, userFirstName text, userLastName text,
                        PRIMARY KEY(songTitle, userId))
                        """)

# EXECUTE QUERY
try:
    session.execute(musicApp_table_query)
except Exception as e:  print(e)
                    

In [59]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        musicApp_table_query  = ("""INSERT INTO music_app_username 
                                (songTitle,userId, userFirstName, userLastName)
                                VALUES (%s, %s, %s, %s)
                                """)
        
        
        
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(musicApp_table_query, (line[9], int(line[10]), line[1],line[4]))

In [60]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

musicApp_table_query = """
SELECT userFirstName, userLastName, songTitle
FROM music_app_username
WHERE songTitle = 'All Hands Against His Own'
"""


try:
    rows = session.execute(musicApp_table_query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(userfirstname='Jacqueline', userlastname='Lynch', songtitle='All Hands Against His Own')
Row(userfirstname='Tegan', userlastname='Levine', songtitle='All Hands Against His Own')
Row(userfirstname='Sara', userlastname='Johnson', songtitle='All Hands Against His Own')


### Drop the tables before closing out the sessions

In [61]:
## TO-DO: Drop the table before closing out the sessions

In [62]:
query = "drop table music_app_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table music_app_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table music_app_username"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)



### Close the session and cluster connection¶

In [63]:
session.shutdown()
cluster.shutdown()

In [64]:
"""

/*************************************************
* Title: <BrittaInData/Data_Engineering>
* Author: <Britta Bettendorf>
* Date: <12/17/2020>
* Code version: <Python>
* Availability: <https://github.com/BrittaInData/Data_Engineering/tree/master/
*************************************************/

"""

'\n\n/*************************************************\n* Title: <BrittaInData/Data_Engineering>\n* Author: <Britta Bettendorf>\n* Date: <12/17/2020>\n* Code version: <Python>\n* Availability: <https://github.com/BrittaInData/Data_Engineering/tree/master/\n*************************************************/\n\n'